In [3]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Import our custom DiD implementation
from src.causal_methods.did import DifferenceInDifferences, load_and_prepare_data

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Imports successful!")


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Load the data
df = load_and_prepare_data("../data/simulated_users.csv")

print("Dataset shape:", df.shape)
print("\nKey variables for DiD analysis:")
print("- Treatment: used_smart_assistant")
print("- Pre-treatment outcome: filed_2023") 
print("- Post-treatment outcome: filed_2024")
print("- User identifier: user_id")

# Quick overview
df[['user_id', 'used_smart_assistant', 'filed_2023', 'filed_2024', 'age', 'income_bracket']].head()


In [ ]:
# Summary statistics by treatment group
print("="*60)
print("TREATMENT VS CONTROL GROUP COMPARISON")
print("="*60)

summary_stats = df.groupby('used_smart_assistant').agg({
    'filed_2023': ['count', 'mean'],
    'filed_2024': 'mean',
    'age': 'mean',
    'tech_savviness': 'mean'
}).round(3)

print("\nSummary by Treatment Status:")
print(summary_stats)

# Calculate naive treatment effect
treated_2024 = df[df['used_smart_assistant'] == 1]['filed_2024'].mean()
control_2024 = df[df['used_smart_assistant'] == 0]['filed_2024'].mean()
naive_effect = treated_2024 - control_2024

print(f"\n📈 NAIVE COMPARISON (2024 only):")
print(f"Treated group filing rate: {treated_2024:.1%}")
print(f"Control group filing rate: {control_2024:.1%}")
print(f"Naive treatment effect: {naive_effect:.1%}")
print(f"\n⚠️  This is likely BIASED due to selection effects!")


In [ ]:
# Create means table for visualization
means_table = pd.DataFrame({
    'Group': ['Control (No Smart Assistant)', 'Treated (Used Smart Assistant)'],
    '2023 (Pre-treatment)': [
        df[df['used_smart_assistant'] == 0]['filed_2023'].mean(),
        df[df['used_smart_assistant'] == 1]['filed_2023'].mean()
    ],
    '2024 (Post-treatment)': [
        df[df['used_smart_assistant'] == 0]['filed_2024'].mean(),
        df[df['used_smart_assistant'] == 1]['filed_2024'].mean()
    ]
})

print("Filing Rates by Group and Year:")
print(means_table.round(3))

# Calculate the changes
control_change = means_table.loc[0, '2024 (Post-treatment)'] - means_table.loc[0, '2023 (Pre-treatment)']
treated_change = means_table.loc[1, '2024 (Post-treatment)'] - means_table.loc[1, '2023 (Pre-treatment)']

print(f"\n📊 CHANGES OVER TIME:")
print(f"Control group change: {control_change:.3f}")
print(f"Treated group change: {treated_change:.3f}")
print(f"Difference-in-Differences: {treated_change - control_change:.3f}")

# Visualize parallel trends
fig, ax = plt.subplots(figsize=(10, 6))

years = [2023, 2024]
control_means = [means_table.loc[0, '2023 (Pre-treatment)'], means_table.loc[0, '2024 (Post-treatment)']]
treated_means = [means_table.loc[1, '2023 (Pre-treatment)'], means_table.loc[1, '2024 (Post-treatment)']]

ax.plot(years, control_means, 'o-', linewidth=3, markersize=10, 
        label='Control (No Smart Assistant)', color='red')
ax.plot(years, treated_means, 's-', linewidth=3, markersize=10, 
        label='Treated (Used Smart Assistant)', color='blue')

# Add counterfactual line (what treated would have been without treatment)
counterfactual_2024 = means_table.loc[1, '2023 (Pre-treatment)'] + control_change
ax.plot([2023, 2024], [means_table.loc[1, '2023 (Pre-treatment)'], counterfactual_2024], 
        '--', linewidth=2, color='lightblue', alpha=0.7, label='Counterfactual (Treated without treatment)')

# Add treatment effect arrow
ax.annotate('', xy=(2024, means_table.loc[1, '2024 (Post-treatment)']), 
            xytext=(2024, counterfactual_2024),
            arrowprops=dict(arrowstyle='<->', color='green', lw=3))
ax.text(2024.05, (means_table.loc[1, '2024 (Post-treatment)'] + counterfactual_2024)/2, 
        f'DiD Effect\n{treated_change - control_change:.3f}', 
        fontsize=12, color='green', weight='bold')

# Formatting
ax.axvline(x=2023.5, color='gray', linestyle='--', alpha=0.7, label='Smart Assistant Launch')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Filing Rate', fontsize=12)
ax.set_title('Parallel Trends: Filing Rates Over Time', fontsize=14, weight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xticks(years)
ax.set_ylim(0.75, 0.95)

plt.tight_layout()
plt.show()


In [ ]:
# Initialize DiD estimator
did_estimator = DifferenceInDifferences(df)

# Prepare panel data (reshape from wide to long)
panel_data = did_estimator.prepare_panel_data()

print("Panel data structure:")
print(f"Original data: {len(df)} users")
print(f"Panel data: {len(panel_data)} observations ({len(panel_data)//2} users × 2 time periods)")

# Show structure of panel data
print("\nPanel data sample:")
panel_data.head(10)


In [ ]:
# Estimate DiD effect (basic model)
print("🔧 BASIC DiD MODEL")
print("="*50)

basic_results = did_estimator.estimate_did()

# Display results
print(f"DiD Treatment Effect: {basic_results['did_estimate']:.4f}")
print(f"Standard Error: {basic_results['standard_error']:.4f}")
print(f"P-value: {basic_results['p_value']:.4f}")
print(f"95% Confidence Interval: [{basic_results['conf_int_lower']:.4f}, {basic_results['conf_int_upper']:.4f}]")

# Show the full regression output
print("\n📊 FULL REGRESSION RESULTS:")
print(basic_results['model'].summary())


In [ ]:
# Estimate DiD with control variables
print("🔧 DiD MODEL WITH CONTROLS")
print("="*50)

control_vars = ['age', 'tech_savviness']
controlled_results = did_estimator.estimate_did(control_vars=control_vars)

print(f"DiD Treatment Effect (with controls): {controlled_results['did_estimate']:.4f}")
print(f"Standard Error: {controlled_results['standard_error']:.4f}")
print(f"P-value: {controlled_results['p_value']:.4f}")
print(f"95% Confidence Interval: [{controlled_results['conf_int_lower']:.4f}, {controlled_results['conf_int_upper']:.4f}]")

# Compare with basic model
print(f"\n📈 COMPARISON:")
print(f"Basic DiD estimate: {basic_results['did_estimate']:.4f}")
print(f"Controlled DiD estimate: {controlled_results['did_estimate']:.4f}")
print(f"Difference: {controlled_results['did_estimate'] - basic_results['did_estimate']:.4f}")

# Update main results to the controlled version
did_estimator.results['main'] = controlled_results


In [ ]:
# Test parallel trends assumption
parallel_trends_results = did_estimator.parallel_trends_test()

print("🔍 PARALLEL TRENDS TESTS")
print("="*50)
print(f"Test type: {parallel_trends_results['test_type']}")
print(f"Description: {parallel_trends_results['description']}")
print(f"Interpretation: {parallel_trends_results['interpretation']}")

print("\nCorrelations between treatment and pre-treatment outcomes:")
for outcome, result in parallel_trends_results['results'].items():
    significance = "***" if result['significant'] else ""
    print(f"  {outcome}: {result['correlation']:.3f} (p={result['p_value']:.3f}) {significance}")

print("\n⚠️  High correlations suggest potential parallel trends violations")
print("✅  Low correlations support the parallel trends assumption")


In [ ]:
# Estimate effects by age group
age_effects = did_estimator.estimate_heterogeneous_effects('age_group')

print("🎯 HETEROGENEOUS EFFECTS BY AGE GROUP")
print("="*50)

for age_group, results in age_effects['results'].items():
    significance = "***" if results['p_value'] < 0.001 else ("**" if results['p_value'] < 0.01 else ("*" if results['p_value'] < 0.05 else ""))
    print(f"{age_group}: {results['did_estimate']:.4f} ({results['standard_error']:.4f}) {significance}")
    print(f"  95% CI: [{results['conf_int_lower']:.4f}, {results['conf_int_upper']:.4f}]")
    print(f"  N users: {results['n_users']}")
    print()

# Plot heterogeneous effects
fig = did_estimator.plot_subgroup_effects(age_effects, figsize=(12, 6))
plt.show()


In [ ]:
# Effects by income bracket
income_effects = did_estimator.estimate_heterogeneous_effects('income_bracket')

print("🎯 HETEROGENEOUS EFFECTS BY INCOME BRACKET")
print("="*50)

for income_bracket, results in income_effects['results'].items():
    significance = "***" if results['p_value'] < 0.001 else ("**" if results['p_value'] < 0.01 else ("*" if results['p_value'] < 0.05 else ""))
    print(f"{income_bracket}: {results['did_estimate']:.4f} ({results['standard_error']:.4f}) {significance}")
    print(f"  95% CI: [{results['conf_int_lower']:.4f}, {results['conf_int_upper']:.4f}]")
    print()

# Plot income effects
fig = did_estimator.plot_subgroup_effects(income_effects, figsize=(12, 6))
plt.show()


In [ ]:
# Effects by user type (new vs returning)
user_type_effects = did_estimator.estimate_heterogeneous_effects('user_type')

print("🎯 HETEROGENEOUS EFFECTS BY USER TYPE")
print("="*50)

for user_type, results in user_type_effects['results'].items():
    significance = "***" if results['p_value'] < 0.001 else ("**" if results['p_value'] < 0.01 else ("*" if results['p_value'] < 0.05 else ""))
    print(f"{user_type}: {results['did_estimate']:.4f} ({results['standard_error']:.4f}) {significance}")
    print(f"  95% CI: [{results['conf_int_lower']:.4f}, {results['conf_int_upper']:.4f}]")
    print()

# Plot user type effects
fig = did_estimator.plot_subgroup_effects(user_type_effects, figsize=(10, 4))
plt.show()


In [ ]:
# Generate comprehensive summary
summary = did_estimator.summary_report()
print(summary)
